In [1]:
import random
import requests
from datetime import datetime, timedelta
from concurrent.futures import ProcessPoolExecutor
import json

In [2]:
# api_key = '337586e7326dcb828d7a386379093040'
# api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'

In [3]:
# Function to get current weather
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

# Function to get weather forecast
def get_forecast(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

# Generate random future date within the next 5 days
def generate_random_future_date():
    days = random.randint(1, 5)
    future_date = datetime.now() + timedelta(days=days)
    return future_date.strftime('%Y-%m-%d')

# Function to create a user prompt and intent extraction
def create_user_prompt_and_intent(dummy):
    cities = [
        "Kolkata", "New York", "London", "Tokyo", "Sydney", "Paris", "Berlin", "Toronto",
        "Mumbai", "Shanghai", "Los Angeles", "Chicago", "Houston", "Phoenix", "Philadelphia",
        "San Antonio", "San Diego", "Dallas", "San Jose", "Austin", "Beijing", "Moscow",
        "Bangkok", "Cairo", "Istanbul", "Buenos Aires", "Rio de Janeiro", "Lagos", "Lima",
        "Jakarta", "Karachi", "Santiago", "Seoul", "Mexico City", "São Paulo", "Dubai",
        "Singapore", "Hong Kong", "Kuala Lumpur", "Tehran", "Baghdad", "Hanoi", "Riyadh",
        "Cape Town", "Nairobi", "Casablanca", "Accra", "Addis Ababa", "Ho Chi Minh City",
        "Manila", "Dhaka", "Abu Dhabi", "Ankara", "Brisbane", "Barcelona", "Madrid",
        "Rome", "Vienna", "Zurich", "Copenhagen", "Oslo", "Stockholm", "Helsinki", "Athens",
        "Budapest", "Warsaw", "Prague", "Brussels", "Amsterdam", "Dublin", "Edinburgh",
        "Glasgow", "Lisbon", "Munich", "Frankfurt", "Hamburg", "Stuttgart", "Lyon", "Marseille",
        "Nice", "Bordeaux", "Toulouse", "Venice", "Florence", "Naples", "Milan", "Turin",
        "Bologna", "Palermo", "Athens", "Thessaloniki", "Osaka", "Nagoya", "Kyoto", "Fukuoka",
        "Sapporo", "Sendai", "Yokohama", "Kobe", "Hiroshima"
    ]
    info_requests = ["current_weather", "forecast_weather"]
    dates = ["today", "tomorrow"] + [generate_random_future_date() for _ in range(8)]
    times_of_day = ["morning", "afternoon", "evening", "night", ""]

    city = random.choice(cities)
    info_request = random.choice(info_requests)
    date = random.choice(dates) if info_request == "forecast_weather" else "today"
    time_of_day = random.choice(times_of_day)

    if time_of_day:
        user_prompt = f"What is the {info_request.replace('_', ' ')} in {city} {date} {time_of_day}?"
        intent_extraction = {
            "intent": info_request,
            "entities": {
                "city": city,
                "date": date,
                "time_of_day": time_of_day
            }
        }
    else:
        user_prompt = f"What is the {info_request.replace('_', ' ')} in {city} {date}?"
        intent_extraction = {
            "intent": info_request,
            "entities": {
                "city": city,
                "date": date
            }
        }

    return {
        "user_input": user_prompt,
        "intent_extraction": intent_extraction
    }

# Generate the intent identification dataset
def generate_intent_dataset(num_samples=800):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(create_user_prompt_and_intent, range(num_samples)))
    return results

intent_dataset = generate_intent_dataset()

In [5]:
# intent_dataset

In [8]:
with open('data/intent_dataset1.json', 'w') as f:
    json.dump(intent_dataset, f, indent=4)

In [10]:
len(intent_dataset)

800

In [6]:
def extract_forecast_for_time(forecast_list, date, time_of_day):
    time_map = {
        "morning": range(6, 12),
        "afternoon": range(12, 18),
        "evening": range(18, 24),
        "night": range(0, 6),
    }

    if time_of_day in time_map:
        filtered_forecasts = [
            forecast for forecast in forecast_list 
            if date in forecast['dt_txt'] and datetime.strptime(forecast['dt_txt'], '%Y-%m-%d %H:%M:%S').hour in time_map[time_of_day]
        ]
    else:
        filtered_forecasts = [
            forecast for forecast in forecast_list if date in forecast['dt_txt']
        ]

    return filtered_forecasts

def create_response_dataset_entry(intent_entry):
    city = intent_entry["intent_extraction"]["entities"]["city"]
    info_request = intent_entry["intent_extraction"]["intent"]
    date = intent_entry["intent_extraction"]["entities"]["date"]
    time_of_day = intent_entry["intent_extraction"]["entities"].get("time_of_day", "")
    user_prompt = intent_entry["user_input"]

    if info_request == "current_weather":
        api_response = get_weather(city)
        if api_response:
            weather = api_response['weather'][0]['description']
            temp = api_response['main']['temp']
            wind_speed = api_response['wind']['speed']
            humidity = api_response['main']['humidity']

            generated_response = (
                f"The current weather in {city} {time_of_day} is {weather} with a temperature of {temp}°C, "
                f"wind speed of {wind_speed} meters per second, and humidity of {humidity}%."
            )

            return {
                "user_input": user_prompt,
                "api_response": {
                    "location": f"{city}, {api_response['sys']['country']}",
                    "temperature": temp,
                    "description": weather,
                    "wind_speed": wind_speed,
                    "humidity": humidity
                },
                "assistant_response": generated_response
            }

    elif info_request == "forecast_weather":
        api_response = get_forecast(city)
        if api_response:
            forecast_list = api_response['list']
            filtered_forecasts = extract_forecast_for_time(forecast_list, date, time_of_day)
            if filtered_forecasts:
                selected_forecast = filtered_forecasts[0]
                forecast_temp = selected_forecast['main']['temp']
                forecast_weather = selected_forecast['weather'][0]['description']
                wind_speed = selected_forecast['wind']['speed']
                humidity = selected_forecast['main']['humidity']

                generated_response = (
                    f"The forecast for {city} on {date} {time_of_day} is {forecast_weather} with a temperature of {forecast_temp}°C, "
                    f"wind speed of {wind_speed} meters per second, and humidity of {humidity}%."
                )

                return {
                    "user_input": user_prompt,
                    "api_response": {
                        "date": date,
                        "temperature": forecast_temp,
                        "description": forecast_weather,
                        "wind_speed": wind_speed,
                        "humidity": humidity
                    },
                    "assistant_response": generated_response
                }

# Generate the response generation dataset
def generate_response_dataset(intent_dataset):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(create_response_dataset_entry, intent_dataset))
    return [result for result in results if result is not None]

In [7]:
response_dataset = generate_response_dataset(intent_dataset)

In [9]:
with open('data/response_dataset1.json', 'w') as f:
    json.dump(response_dataset, f, indent=4)

In [9]:
len(response_dataset)

689

In [11]:
response_dataset

[{'user_input': 'What is the forecast weather in Zurich 2024-06-15 morning?',
  'api_response': {'date': '2024-06-15',
   'temperature': 12.62,
   'description': 'light rain',
   'wind_speed': 2.13,
   'humidity': 97},
  'assistant_response': 'The forecast for Zurich on 2024-06-15 morning is light rain with a temperature of 12.62°C, wind speed of 2.13 meters per second, and humidity of 97%.'},
 {'user_input': 'What is the forecast weather in Naples 2024-06-18 morning?',
  'api_response': {'date': '2024-06-18',
   'temperature': 25.89,
   'description': 'clear sky',
   'wind_speed': 0.86,
   'humidity': 56},
  'assistant_response': 'The forecast for Naples on 2024-06-18 morning is clear sky with a temperature of 25.89°C, wind speed of 0.86 meters per second, and humidity of 56%.'},
 {'user_input': 'What is the forecast weather in Palermo 2024-06-14 morning?',
  'api_response': {'date': '2024-06-14',
   'temperature': 18.28,
   'description': 'scattered clouds',
   'wind_speed': 3.35,
  